In [1]:
from metatrader.utils.factory import MetatraderFactory
metatrader = MetatraderFactory.get_metatrader()
import pandas as pd
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go

mt5 = metatrader.connect()

symbol = 'EURUSD'
timeframe = mt5.TIMEFRAME_M5

trade_volume = 500000
commission = 0.000025



Connected to Metatrader successfully.


In [2]:
bars = pd.DataFrame(mt5.copy_rates_from_pos(symbol, timeframe, 0,5000))
bars['time'] = pd.to_datetime(bars['time'], unit='s')
bars['trade_volume'] = trade_volume
bars

,time,open,high,low,close,tick_volume,spread,real_volume,trade_volume
0,2023-09-29 15:35:00,1.05896,1.06062,1.05883,1.06048,917,0,0,500000
1,2023-09-29 15:40:00,1.06048,1.06101,1.06020,1.06046,719,0,0,500000
2,2023-09-29 15:45:00,1.06043,1.06047,1.05989,1.06003,576,0,0,500000
3,2023-09-29 15:50:00,1.06002,1.06041,1.05992,1.06008,561,0,0,500000
4,2023-09-29 15:55:00,1.06012,1.06038,1.05966,1.05998,517,0,0,500000
...,...,...,...,...,...,...,...,...,...
4995,2023-10-25 00:30:00,1.05923,1.05923,1.05913,1.05921,45,22,0,500000
4996,2023-10-25 00:35:00,1.05920,1.05928,1.05916,1.05921,109,8,0,500000
4997,2023-10-25 00:40:00,1.05923,1.05929,1.05919,1.05923,96,20,0,500000
4998,2023-10-25 00:45:00,1.05924,1.05925,1.05909,1.05913,143,8,0,500000


In [3]:
bars['ema_20'] = bars['close'].ewm(span=20).mean()
bars['ema_10'] = bars['close'].ewm(span=10).mean()
bars['hour'] = bars['time'].dt.hour
bars

,time,open,high,low,close,tick_volume,spread,real_volume,trade_volume,ema_20,ema_10,hour
0,2023-09-29 15:35:00,1.05896,1.06062,1.05883,1.06048,917,0,0,500000,1.060480,1.060480,15
1,2023-09-29 15:40:00,1.06048,1.06101,1.06020,1.06046,719,0,0,500000,1.060469,1.060469,15
2,2023-09-29 15:45:00,1.06043,1.06047,1.05989,1.06003,576,0,0,500000,1.060308,1.060293,15
3,2023-09-29 15:50:00,1.06002,1.06041,1.05992,1.06008,561,0,0,500000,1.060242,1.060223,15
4,2023-09-29 15:55:00,1.06012,1.06038,1.05966,1.05998,517,0,0,500000,1.060179,1.060153,15
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2023-10-25 00:30:00,1.05923,1.05923,1.05913,1.05921,45,22,0,500000,1.059103,1.059130,0
4996,2023-10-25 00:35:00,1.05920,1.05928,1.05916,1.05921,109,8,0,500000,1.059113,1.059145,0
4997,2023-10-25 00:40:00,1.05923,1.05929,1.05919,1.05923,96,20,0,500000,1.059124,1.059160,0
4998,2023-10-25 00:45:00,1.05924,1.05925,1.05909,1.05913,143,8,0,500000,1.059125,1.059155,0


In [4]:
bars2 = bars.dropna().copy()

def get_signal(x):

    if 9 <= x['hour'] <= 22:

        if x['ema_10'] > x['ema_20']:
            x["position"] = "long"
            return 1
        elif x['ema_10'] < x['ema_20']:
            x["position"] = "short"
            return -1

    return 0

bars2['signal'] = bars2.apply(get_signal, axis=1)
bars2['prev_price'] = bars2['close'].shift(1)
bars2['price_change'] = bars2['close'] - bars2['prev_price']

bars2['signal_change'] = (bars2['signal'] - bars2['signal'].shift(1)).abs()
bars2['commission'] = bars2.apply(lambda x: commission * x['signal_change'] * x['trade_volume'], axis=1)
bars2

,time,open,high,low,close,tick_volume,spread,real_volume,trade_volume,ema_20,ema_10,hour,signal,prev_price,price_change,signal_change,commission
0,2023-09-29 15:35:00,1.05896,1.06062,1.05883,1.06048,917,0,0,500000,1.060480,1.060480,15,0,NaN,NaN,NaN,NaN
1,2023-09-29 15:40:00,1.06048,1.06101,1.06020,1.06046,719,0,0,500000,1.060469,1.060469,15,-1,1.06048,-0.00002,1.0,12.5
2,2023-09-29 15:45:00,1.06043,1.06047,1.05989,1.06003,576,0,0,500000,1.060308,1.060293,15,-1,1.06046,-0.00043,0.0,0.0
3,2023-09-29 15:50:00,1.06002,1.06041,1.05992,1.06008,561,0,0,500000,1.060242,1.060223,15,-1,1.06003,0.00005,0.0,0.0
4,2023-09-29 15:55:00,1.06012,1.06038,1.05966,1.05998,517,0,0,500000,1.060179,1.060153,15,-1,1.06008,-0.00010,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2023-10-25 00:30:00,1.05923,1.05923,1.05913,1.05921,45,22,0,500000,1.059103,1.059130,0,0,1.05918,0.00003,0.0,0.0
4996,2023-10-25 00:35:00,1.05920,1.05928,1.05916,1.05921,109,8,0,500000,1.059113,1.059145,0,0,1.05921,0.00000,0.0,0.0
4997,2023-10-25 00:40:00,1.05923,1.05929,1.05919,1.05923,96,20,0,500000,1.059124,1.059160,0,0,1.05921,0.00002,0.0,0.0
4998,2023-10-25 00:45:00,1.05924,1.05925,1.05909,1.05913,143,8,0,500000,1.059125,1.059155,0,0,1.05923,-0.00010,0.0,0.0


In [5]:
bars3 = bars2.dropna().copy()
bars3 = bars3.loc[bars3['signal_change']!=0]
bars3['profit'] = bars3['signal'] * bars3['price_change'] * bars3['trade_volume']
bars3['gross_profit'] = bars3['profit'].cumsum()
bars3['net_profit'] = bars3['gross_profit'] - bars3['commission'].cumsum()


bars3

,time,open,high,low,close,tick_volume,spread,real_volume,trade_volume,ema_20,ema_10,hour,signal,prev_price,price_change,signal_change,commission,profit,gross_profit,net_profit
1,2023-09-29 15:40:00,1.06048,1.06101,1.06020,1.06046,719,0,0,500000,1.060469,1.060469,15,-1,1.06048,-0.00002,1.0,12.5,10.0,10.0,-2.5
41,2023-09-29 19:00:00,1.05834,1.05851,1.05820,1.05839,242,0,0,500000,1.058111,1.058120,19,1,1.05833,0.00006,2.0,25.0,30.0,40.0,2.5
43,2023-09-29 19:10:00,1.05828,1.05837,1.05781,1.05781,239,0,0,500000,1.058097,1.058087,19,-1,1.05828,-0.00047,2.0,25.0,235.0,275.0,212.5
45,2023-09-29 19:20:00,1.05795,1.05846,1.05787,1.05838,243,0,0,500000,1.058112,1.058120,19,1,1.05795,0.00043,2.0,25.0,215.0,490.0,402.5
50,2023-09-29 19:45:00,1.05799,1.05848,1.05757,1.05761,333,0,0,500000,1.058114,1.058090,19,-1,1.05800,-0.00039,2.0,25.0,195.0,685.0,572.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4827,2023-10-24 10:30:00,1.06772,1.06772,1.06601,1.06678,1131,0,0,500000,1.068185,1.068088,10,-1,1.06776,-0.00098,2.0,25.0,490.0,24020.0,20282.5
4879,2023-10-24 14:50:00,1.06356,1.06415,1.06356,1.06411,289,0,0,500000,1.063448,1.063473,14,1,1.06356,0.00055,2.0,25.0,275.0,24295.0,20532.5
4881,2023-10-24 15:00:00,1.06353,1.06365,1.06287,1.06291,490,0,0,500000,1.063404,1.063379,15,-1,1.06353,-0.00062,2.0,25.0,310.0,24605.0,20817.5
4973,2023-10-24 22:40:00,1.05882,1.05900,1.05880,1.05899,76,0,0,500000,1.058883,1.058888,22,1,1.05883,0.00016,2.0,25.0,80.0,24685.0,20872.5


In [6]:
px.line(bars3, 'time', ['gross_profit', 'net_profit'], title="SP500 NY session Backtest EMA Trendfollowing")


c:\Users\praveenjaisankar\AppData\Local\pypoetry\Cache\virtualenvs\metatrader-Cdlomroj-py3.10\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()
